In [1]:
%load_ext dotenv
%dotenv

In [2]:
import os
import json
import requests
import contextlib

import sqlalchemy as sa
from sqlalchemy.orm import sessionmaker

import pandas as pd
import numpy as np

import re
import urllib.parse

import matplotlib.pyplot as plt

In [3]:
CONFIG_PATH = "config.json"
OUT_FILE = "traintest.pq"
VERBOSE = False

In [4]:
CONFIG = None
ENGINES = {}
TABLES = {}
BINDS = {}
SESSION = None


def config_template():
    default_conn = {
        "dialect": "postgresql",
        "host": "localhost",
        "port": 5432,
        "dbname": "INVALID",
        "schema": "public",
        "user": "INVALID",
        "passwd": "INVALID",
    }
    return {
        "dbs": {
            "login": default_conn.copy(),
            "sm": default_conn.copy(),
            "exp": default_conn.copy(),
            "ap": default_conn.copy(),
            "blogs": default_conn.copy(),
        }
    }


def get_config():
    global CONFIG
    
    if CONFIG is not None:
        return CONFIG
    if not os.path.exists(CONFIG_PATH):
        with open(CONFIG_PATH, "w") as fout:
            print(json.dumps(config_template(), indent=4, sort_keys=True), file=fout)
        raise ValueError(
            f"config file missing. new file was created at '{CONFIG_PATH}'. "
            "please correct values in file and run again")
    with open(CONFIG_PATH, "r") as fin:
        CONFIG = json.load(fin)
    config_out = {
        "dbs": {
            "login": CONFIG["dbs"]["login"].copy(),
            "sm": CONFIG["dbs"]["sm"].copy(),
            "exp": CONFIG["dbs"]["exp"].copy(),
            "ap": CONFIG["dbs"]["ap"].copy(),
            "blogs": CONFIG["dbs"]["blogs"].copy(),
        }
    }
    config_out["dbs"]["login"]["passwd"] = "..."
    config_out["dbs"]["sm"]["passwd"] = "..."
    config_out["dbs"]["exp"]["passwd"] = "..."
    config_out["dbs"]["ap"]["passwd"] = "..."
    print(f"loaded config\n{json.dumps(config_out, indent=2, sort_keys=True)}")
    return CONFIG


def get_engine(dbname):
    res = ENGINES.get(dbname)
    if res is not None:
        return res
    db = get_config()["dbs"][dbname]
    user = urllib.parse.quote_plus(db["user"])
    passwd = urllib.parse.quote_plus(db["passwd"])
    engine = sa.create_engine(
        f"{db['dialect']}://{user}:{passwd}@{db['host']}:{db['port']}/{db['dbname']}",
        echo=VERBOSE)
    engine = engine.execution_options(
        schema_translate_map={None: db['schema']})
    res = engine, sa.MetaData()
    ENGINES[dbname] = res
    return res


def get_table(dbname, tablename):
    global SESSION
    
    key = (dbname, tablename)
    res = TABLES.get(key)
    if res is not None:
        return res
    SESSION = None
    engine, metadata = get_engine(dbname)
    res = sa.Table(
        tablename,
        metadata,
        autoload_with=engine)
    TABLES[key] = res
    BINDS[res] = engine
    return res


@contextlib.contextmanager
def get_session():
    global SESSION
    
    session = SESSION
    if session is None:
        session = sessionmaker()
        session.configure(binds=BINDS)
        SESSION = session
    with session() as res:
        yield res

In [5]:
# experiments tables
t_exp_pads = get_table("exp", "pads")
t_exp_tagging = get_table("exp", "tagging")

loaded config
{
  "dbs": {
    "ap": {
      "dbname": "action_plans_platform",
      "dialect": "postgresql",
      "host": "acclabs-global-login.postgres.database.azure.com",
      "passwd": "...",
      "port": 5432,
      "schema": "public",
      "user": "acclabshqadmin@acclabs-global-login"
    },
    "blogs": {
      "dbname": "blogs",
      "dialect": "postgresql",
      "host": "acclabs.postgres.database.azure.com",
      "passwd": "acclabsblogs@2023",
      "port": 5432,
      "schema": "public",
      "user": "undpacclab@acclabs"
    },
    "exp": {
      "dbname": "experiments_platform",
      "dialect": "postgresql",
      "host": "acclabs-global-login.postgres.database.azure.com",
      "passwd": "...",
      "port": 5432,
      "schema": "public",
      "user": "acclabshqadmin@acclabs-global-login"
    },
    "login": {
      "dbname": "postgres",
      "dialect": "postgresql",
      "host": "acclabs-global-login.postgres.database.azure.com",
      "passwd": "...",
     

/var/folders/ff/zrdw3xx56dd1w9xbl6x_yl0c0000gn/T/ipykernel_78105/3044908452.py:85: SAWarning: Did not recognize type 'ltree' of column 'version'
  res = sa.Table(


In [6]:
with get_session() as session:
    stmt = sa.select(sa.func.count(t_exp_pads.c.id))
    stmt = stmt.where(t_exp_pads.c.status >= 2)
    exp_pad_count = int(session.execute(stmt).one()[0])
    print(exp_pad_count)

218


In [7]:
with get_session() as session:
    stmt = sa.select(t_exp_pads.c.id, t_exp_pads.c.title, t_exp_pads.c.sections).where(t_exp_pads.c.id == 457)
    res = session.execute(stmt).one()[2]

In [8]:
# print(json.dumps(res, indent=2, sort_keys=True))

In [9]:
def get_section_items(sections, name):
    if sections is None:
        return None
    for elem in sections:
        if elem.get('type') != "section":
            continue
        title = elem.get('title')
        if title is None:
            continue
        if title.lower() == name.lower():
            return elem.get('items', [])
    return None

In [10]:
for elem in res:
    print(elem.get('type'), elem.get('title'), len(elem['items']))

section Overview 13
section None 0
section Design 14
section Results 8
section Learning 10


In [11]:
{item["type"] for item in get_section_items(res, "Design")}

{'checklist', 'embed', 'img', 'radiolist', 'txt'}

In [12]:
def has_content(item, *, only_required):
    try:
        if only_required and not item.get("required", False):
            return True
        itype = item["type"]
        if itype == "txt":
            return bool(item.get("txt", "").strip())
        if itype == "social":
            return bool(item["text"].strip())
        if itype in ("checklist", "radiolist"):
            return any((opt["checked"] for opt in item["options"]))
        if itype in ("img", "embed", "mosaic"):
            return item.get("has_content", False)
        raise ValueError(f"unknown item type: {item}")
    except Exception as e:
        raise ValueError(f"item: {item}") from e

In [13]:
def content_ratio(pid, sections, name):
    try:
        items = get_section_items(sections, name)
        if items is None:
            return 0, 0
        required = sum(has_content(item, only_required=True) for item in items) / len(items)
        extra = sum(has_content(item, only_required=False) for item in items) / len(items)
        return required, extra
    except Exception as e:
        raise ValueError(f"invalid item for pad {pid}") from e

In [14]:
content_ratio(457, res, "Design")

(1.0, 0.7857142857142857)

In [15]:
stats = {
    "id": [],
    "title": [],
    "design_required": [],
    "design_extra": [],
    "results_required": [],
    "results_extra": [],
}
with get_session() as session:
    stmt = sa.select(t_exp_pads.c.id, t_exp_pads.c.title, t_exp_pads.c.sections)
    for row in session.execute(stmt):
        design_required, design_extra = content_ratio(row[0], row[2], "design")
        results_required, results_extra = content_ratio(row[0], row[2], "results")
        stats["id"].append(row[0])
        stats["title"].append(row[1])
        stats["design_required"].append(design_required)
        stats["design_extra"].append(design_extra)
        stats["results_required"].append(results_required)
        stats["results_extra"].append(results_extra)
stats_df = pd.DataFrame(stats)
stats_df

,id,title,design_required,design_extra,results_required,results_extra
0,441,None,0.000000,0.000000,0.0,0.000
1,430,Review #1,0.000000,0.000000,0.0,0.000
2,86,,1.000000,0.000000,1.0,0.000
3,358,Incentive system for recruitment of Covid-Cham...,1.000000,1.000000,1.0,0.875
4,232,Agri Waste to Energy,1.000000,0.857143,1.0,0.125
...,...,...,...,...,...,...
308,235,Promoting Youth Entrepreneurship in Rural Areas,1.000000,0.857143,1.0,0.000
309,237,Japan Innovation challenge:,0.285714,0.000000,1.0,0.000
310,241,Marine Litter: Behavioral Insights Experiment ...,1.000000,1.000000,1.0,1.000
311,252,"Local Convergence: Promoting Agile, Adaptive, ...",1.000000,1.000000,1.0,1.000


In [16]:
stats_df["dr"] = stats_df["design_required"] > 0
stats_df["de"] = stats_df["design_extra"] > 0
stats_df["rr"] = stats_df["results_required"] > 0
stats_df["re"] = stats_df["results_extra"] > 0

In [17]:
stats_df

,id,title,design_required,design_extra,results_required,results_extra,dr,de,rr,re
0,441,None,0.000000,0.000000,0.0,0.000,False,False,False,False
1,430,Review #1,0.000000,0.000000,0.0,0.000,False,False,False,False
2,86,,1.000000,0.000000,1.0,0.000,True,False,True,False
3,358,Incentive system for recruitment of Covid-Cham...,1.000000,1.000000,1.0,0.875,True,True,True,True
4,232,Agri Waste to Energy,1.000000,0.857143,1.0,0.125,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...
308,235,Promoting Youth Entrepreneurship in Rural Areas,1.000000,0.857143,1.0,0.000,True,True,True,False
309,237,Japan Innovation challenge:,0.285714,0.000000,1.0,0.000,True,False,True,False
310,241,Marine Litter: Behavioral Insights Experiment ...,1.000000,1.000000,1.0,1.000,True,True,True,True
311,252,"Local Convergence: Promoting Agile, Adaptive, ...",1.000000,1.000000,1.0,1.000,True,True,True,True


In [18]:
pd.crosstab(stats_df["dr"], stats_df["rr"], rownames=["design"], colnames=["results"])

results,False,True
design,,
False,25,0
True,0,288


In [19]:
pd.crosstab(stats_df["de"], stats_df["re"], rownames=["design"], colnames=["results"])

results,False,True
design,,
False,59,0
True,71,183
